# Distributed Keys

When dealing with cryptography in general, the source of truth you use for encrypting your secrets is usually a single point of failure, in any naïve implementations. This could be the private key used to generate / verify signatures for example. Also, when decentralizing cryptography protocols, traditional ideas of verification must be extended to encompass many parties, with the intention of removing single point failures, not adding more. This goes over the basics of related distribution protocols for secret sharing and validation.

## Shamir's Secret Sharing

This is the foundation on which many distributed key generation (DKG) protocols are based. The punchline is that it allows for $t$ individuals, out of $n$ total, to verify a signature. Not only does this allow for fault tolerance (e.g. node in the network goes down), but allows for a quorum of attestations, improving trustlessness. Maximum security would require $n>\lfloor 2t - 1\rfloor$.

At the core of this concept is the following truth. Given a set $\{(x_i,y_i)\in\mathbb{R}_2\,|\, i\in[0, t]\}$, there exists a UNIQUE polynomial $q(x)$ of degree $t-1$ such that $q(x_i)=y_i$. We then express this polynomial as:
$$
q(x) = a_0 + a_1x+\cdots+a_{t-1}x^{t-1}
$$
where $a_0$ is the secret to be shared. Discretization is then done as $s_i = q(i)$, where $s_i$ is the partial shared secret. The partial signature is therefore $s_iH(m)$, where $H(m)$ is the hashed message in the group $\mathbb{G}_1$. The uniqueness of polynomial interoplation requires knowledge of exactly $t$ of these partial shares to recover the shared secret. Knowledge of only $t-1$ means that, of the candidate $a_0^\prime\in[0,p)$, there will be again only a unique polynomial fitting $(0, a_0^\prime)$ and $(i, a_i)$, each of these $p$ unique polynomials being equally likely, thus maintaining security of the shared secret.

Recovery of the shared secret is done via Lagrange interpolation:
$$
a_0 = q(0) = \sum_{j=0}^{t-1}y_t\prod_{m=0,m\neq j}^{t-1}\frac{x_m}{x_m-x_j}$$

The problems with this, while admittedly simple and extensible, is that it assumes honesty of the participants (at no point is there an ability to verify partial shares), and further that the shared secret is known singuarly at some point in space-time, both at instantiation of the sharing and at the recovery of the shares. To address these issues, we move beyond to ...

## pedersen DKG

Now in this scheme, each participant generates coefficients, and the cooperation of participants is what allows for the collective generation of a secret. The public parameters in this setup are a large prime $p$, a generator $g$ of a subgroup of $\mathbb{Z}_p^\ast$, and another generator $h$ where no one knows $\log_g h$. 

Firstly, each participant $i$ chooses a secret value $a_i\sim U(0, p)\in\mathbb{Z}_p$, as well as a random polynomial $q_i(x)$ of degree $t-1$ in $\mathbb{Z}_p[x]$:
$$
q_i(x) = a_i + a_{i1}x + a_{i2}x^2 + \cdots + a_{i(t-1)}x^{t-1}
$$
The participant then computes $F_{ij} = ga_{ij}$ for $j\in[0, t-1]$, and publicly broadcasts $\{F_ij | j\in[1, t-1]\}$ (remembering that we don't send $j=0$ since that is the $i$-th secret.
> [!NOTE]  
> In practice, each committment $F_{ij}$ is also multiplied by $hr_{ij}$, where $r_{ij}$ are random values to add entropy to the process.

Then, once everyone has sent the $t-1$ values, everyone computes their shares of their secret polynomial $s_{ij} = q_i(j)$, and sends them and a signature on $s_{ij}$ to participants $1..n$. 

A participant is able to verify that a share received from $j$ is consistent by assuring:
$$
gs_{ji} = \prod_{k=0}^{t-1}F^{i^k}_{jk}
$$
If this fails, then participant $i$ complains publically. If there are complaints, the accused participant must broadcast the correct share (and random value if used). If this fails, they're booted (you're fired!). In this way, we can remove the assumption in Shamir's secret sharing of honesty of the participants, because we can verify when they're lying.

Finally, participant $i$ determines their share as $s_i = \sum_{j=1}^n s_{ji}$, and sign the message. They can also now determine the partials of others. How? Since $\tilde{q}(x)=\sum_{i}^nq_i(x)$ satisfies $s_i=\tilde{q}(i)$, each $s_i$ is a share of $\tilde{q}(0) = x$ (because of SSS!). The participant $j$ can now see the partial of $i$:
$$
gs_i = \prod_{j=1}^ngs_{ji}
$$

What might this look like? I'd recommend looking at the implementation [here](https://docs.rs/secret_sharing_and_dkg/latest/src/secret_sharing_and_dkg/pedersen_dvss.rs.html#1-341) for the basics.
